In [1]:
import pandas as pd
import os
from dotenv import load_dotenv
from neo4j import GraphDatabase

from langchain_openai import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
import gradio as gr
import time

## Setup Connection to Neo4j

In [2]:
if os.path.exists('credentials.env'):
    load_dotenv('credentials.env', override=True)

    # Neo4j
    uri = os.getenv('NEO4J_URI')
    username = os.getenv('NEO4J_USERNAME')
    password = os.getenv('NEO4J_PASSWORD')
    database = os.getenv('NEO4J_DATABASE')

    # AI
    OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
    os.environ['OPENAI_API_KEY']=OPENAI_API_KEY
else:
    print("File 'credentials.env' not found.")

In [3]:
class App:
    def __init__(self, uri, user, password, database=None):
        self.driver = GraphDatabase.driver(uri, auth=(user, password), database=database)
        self.database = database

    def close(self):
        self.driver.close()

    def query(self, query):
        return self.driver.execute_query(query)

    def query_params(self, query, parameters):
        return self.driver.execute_query(query, parameters_=parameters)

    def count_nodes_in_db(self):
        query = "MATCH (n) RETURN COUNT(n)"
        result = self.query(query)
        (key, value) = result.records[0].items()[0]
        return value
        
    def count_nodes_with_label_in_db(self, label):
        query = f"MATCH (n:{label}) RETURN COUNT(n)"
        result = self.query(query)
        (key, value) = result.records[0].items()[0]
        return value
    
    def remove_nodes_relationships(self):
        query ="""
            CALL apoc.periodic.iterate(
                "MATCH (c) RETURN c",
                "WITH c DETACH DELETE c",
                {batchSize: 1000}
            )
        """
        result = self.query(query)

    def remove_all_constraints(self):
        query ="""
            CALL apoc.schema.assert({}, {})
        """
        result = self.query(query)

In [4]:
app = App(uri, username, password, database)

In [5]:
app.count_nodes_in_db()

5951

### Create Vector Index

In [6]:
query = """
    CALL apoc.periodic.iterate(
        "MATCH (b:Beer) RETURN b",
        "WITH b CALL db.create.setNodeVectorProperty(b, 'embedding', b.embedding)",
        {batchSize: 1000}
    )
"""

In [7]:
app.query(query)

EagerResult(records=[<Record batches=3 total=2061 timeTaken=0 committedOperations=2061 failedOperations=0 failedBatches=0 retries=0 errorMessages={} batch={'total': 3, 'errors': {}, 'committed': 3, 'failed': 0} operations={'total': 2061, 'errors': {}, 'committed': 2061, 'failed': 0} wasTerminated=False failedParams={} updateStatistics={'relationshipsDeleted': 0, 'relationshipsCreated': 0, 'nodesDeleted': 0, 'nodesCreated': 0, 'labelsRemoved': 0, 'labelsAdded': 0, 'propertiesSet': 0}>], summary=<neo4j._work.summary.ResultSummary object at 0x30d9ebbd0>, keys=['batches', 'total', 'timeTaken', 'committedOperations', 'failedOperations', 'failedBatches', 'retries', 'errorMessages', 'batch', 'operations', 'wasTerminated', 'failedParams', 'updateStatistics'])

In [8]:
query = """
    CREATE VECTOR INDEX `description-embeddings` IF NOT EXISTS
    FOR (b:Beer) ON (b.embedding)
    OPTIONS {
        indexConfig: {
            `vector.dimensions`: 256,
            `vector.similarity_function`: 'cosine'
        } 
    }
"""

In [9]:
app.query(query)

EagerResult(records=[], summary=<neo4j._work.summary.ResultSummary object at 0x30db0a0d0>, keys=[])

## Setup Chatbot

In [10]:
embedding_model = 'text-embedding-3-small'

In [11]:
embedding_model = OpenAIEmbeddings(
    model=embedding_model,
    openai_api_key=OPENAI_API_KEY,
    dimensions=256
)

In [12]:
llm = ChatOpenAI(temperature=0)
llm.model_name

/var/folders/l0/vn55w75s41703t1w811dtf080000gp/T/ipykernel_88081/3847375981.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(temperature=0)


'gpt-3.5-turbo'

In [13]:
def get_context_vector_search(search_prompt):
    query_vector = embedding_model.embed_query(search_prompt)
    similarity_query = """ 
        CALL db.index.vector.queryNodes("description-embeddings", 5, $query_vector) YIELD node, score
        WITH node as beer, score ORDER BY score DESC
        RETURN beer.name as beer, beer.description as description
       """
    results = app.query_params(similarity_query, {'query_vector': query_vector})    
    scored_beers = "Scored Beers: \n\n" + "\n\n".join(["beer: " + record['beer'] + "\n" + "description: " + record['description'] for record in results.records])    
    return scored_beers

In [14]:
def generate_prompt_vector_search(search_prompt, scored_beers):
    prompt_template = """

    You are a chatbot on beers. Your goal is to help people with questions on beers.  
    A user will come to you with questions on any beers. Their questions must be answered based on the relevant beers found in the score.
    If you make any recommendation. Please explain why based on the descriptions. 

    
    The question is the following: 
    {search_prompt}
    Always respond in the language in which the question was asked. So, do not respond in a different language.
    
    The beers found based on the question are the following: 
    {scored_beers}
    
    """
    prompt = PromptTemplate.from_template(prompt_template)
    
    theprompt = prompt.format_prompt(search_prompt=search_prompt, scored_beers=scored_beers)
    return theprompt

In [15]:
def get_context_graphrag(search_prompt):
    query_vector = embedding_model.embed_query(search_prompt)
    
    similarity_query = """ 
        CALL db.index.vector.queryNodes("description-embeddings", 5, $query_vector) YIELD node, score
        WITH node as beer, score ORDER BY score DESC
        MATCH (beer)-[r:SIMILAR_BEER]-(b2:Beer)
        WITH DISTINCT beer, r, b2
        WHERE r.score > 0.95
        WITH COLLECT(beer) as beers, COLLECT(b2) as similar_beers
        WITH beers + similar_beers AS beers
        UNWIND beers as beer
        WITH DISTINCT beer
        RETURN beer.name as beer, beer.description as description
       """
    results = app.query_params(similarity_query, {'query_vector': query_vector})    
    scored_beers = "Scored Beers: \n\n" + "\n\n".join(["beer: " + record['beer'] + "\n" + "description: " + record['description'] for record in results.records])
    
    return scored_beers

In [16]:
def generate_prompt_graphrag(search_prompt, scored_beers):
    prompt_template = """

    You are a chatbot on beers. Your goal is to help people with questions on beers.  
    A user will come to you with questions on any beers. Their questions must be answered based on the relevant beers found in the score.
    If you make any recommendation. Please explain why based on the descriptions. 

    
    The question is the following: 
    {search_prompt}
    Always respond in the language in which the question was asked. So, do not respond in a different language.
    
    The beers found based on the question are the following: 
    {scored_beers}
    
    """
    prompt = PromptTemplate.from_template(prompt_template)
    
    theprompt = prompt.format_prompt(search_prompt=search_prompt, scored_beers=scored_beers)
    return theprompt

In [17]:
search_prompt = 'I like a very fresh blond beer. Can you recommend me any?'

scored_beers = get_context_vector_search(search_prompt)
print(scored_beers)
theprompt = generate_prompt_vector_search(search_prompt, scored_beers)
llm(theprompt.to_messages()).pretty_print()

Scored Beers: 

beer: 44 Blonde
description: 44 Blonde is a Belgian-style blonde ale that is brewed with a combination of Pilsner malt, wheat, and a touch of aromatic hops. This beer has a light golden color with a slightly hazy appearance and a fluffy white head. It offers a balanced flavor profile with notes of citrus, floral hops, and a subtle sweetness from the malt. The taste is crisp and refreshing, with a medium body and a moderate level of carbonation. Overall, 44 Blonde is a smooth and easy-drinking beer that is perfect for those looking for a classic Belgian ale experience.

beer: Ultra Blonde
description: Ultra Blonde is a Belgian-style blonde ale that is brewed with a combination of Pilsner malt, wheat, and a touch of aromatic hops. This beer is known for its light and refreshing character, with a crisp and clean taste. It has a pale golden color and a subtle fruity aroma with hints of citrus and spice. The flavor profile is balanced, offering a delicate sweetness from the 

/var/folders/l0/vn55w75s41703t1w811dtf080000gp/T/ipykernel_88081/1928929303.py:6: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  llm(theprompt.to_messages()).pretty_print()


================================== Ai Message ==================================

Based on your preference for a very fresh blond beer, I would recommend the Ultra Blonde. This beer is known for its light and refreshing character, with a crisp and clean taste. It has a subtle fruity aroma with hints of citrus and spice, offering a delicate sweetness from the malt balanced with a gentle bitterness from the hops. The smooth, dry finish of Ultra Blonde makes it a perfect choice for those looking for a sessionable and easy-drinking beer that is sure to satisfy your craving for a fresh blond beer experience.


In [18]:
search_prompt = "I like a beer with a fruity taste. Can you recommend me some?"

scored_beers = get_context_vector_search(search_prompt)
print(scored_beers)
theprompt = generate_prompt_vector_search(search_prompt, scored_beers)
llm(theprompt.to_messages()).pretty_print()

Scored Beers: 

beer: Belgian Pêches
description: Belgian Pêches is a refreshing fruit beer brewed with ripe peaches. This beer falls under the category of fruit beer and is known for its sweet and juicy peach flavor that is balanced with a subtle tartness. The aroma is filled with the scent of fresh peaches, while the taste offers a delightful combination of fruity sweetness and a hint of sourness. The beer has a light to medium body with a smooth mouthfeel, making it a perfect choice for those who enjoy a fruity and easy-drinking beer.

beer: La Poirette de Fontaine
description: La Poirette de Fontaine is a Belgian-style fruit beer brewed with a blend of traditional and wild yeast strains. This beer is infused with locally sourced pears, which impart a subtle sweetness and delicate fruitiness to the brew. It has a light body and a slightly hazy appearance, with a golden hue reminiscent of ripe pears. The aroma is dominated by notes of fresh pear, along with hints of floral and spicy 

In [19]:
search_prompt = "Okay I really hate beers with coriander. Can you recommend me some that doesn't taste like coriander?"

scored_beers = get_context_vector_search(search_prompt)
print(scored_beers)
theprompt = generate_prompt_vector_search(search_prompt, scored_beers)
llm(theprompt.to_messages()).pretty_print()

Scored Beers: 

beer: Cornet
description: Cornet is a Belgian strong ale that is brewed with a combination of pale malts, aromatic hops, and a special yeast strain. This beer has a rich golden color and a creamy white head. It offers a complex flavor profile with notes of caramel, honey, and a subtle spiciness. The taste is balanced between sweet malts and a gentle bitterness, with a smooth and slightly dry finish. Cornet has a medium body and a moderate level of carbonation, making it a well-rounded and enjoyable beer for those who appreciate traditional Belgian ales.

beer: Ginder Ale
description: Ginder Ale is a Belgian-style ale brewed with a blend of pale malts, giving it a golden hue and a medium body. This beer is characterized by its spicy and fruity yeast profile, which adds complexity to its flavor profile. It has a slightly sweet malt backbone with notes of citrus, coriander, and a hint of pepper. Ginder Ale has a moderate bitterness and a dry finish, making it a refreshing 

In [20]:
search_prompt = "Okay I really hate beers with coriander. Can you recommend me some that doesn't taste like coriander?"

scored_beers = get_context_graphrag(search_prompt)
print(scored_beers)
theprompt = generate_prompt_graphrag(search_prompt, scored_beers)
llm(theprompt.to_messages()).pretty_print()

Scored Beers: 

beer: Ginder Ale
description: Ginder Ale is a Belgian-style ale brewed with a blend of pale malts, giving it a golden hue and a medium body. This beer is characterized by its spicy and fruity yeast profile, which adds complexity to its flavor profile. It has a slightly sweet malt backbone with notes of citrus, coriander, and a hint of pepper. Ginder Ale has a moderate bitterness and a dry finish, making it a refreshing and easy-drinking beer. It pairs well with a variety of dishes, from grilled meats to salads, making it a versatile choice for any occasion.

beer: Triverius
description: Triverius is a Belgian Tripel-style beer brewed with a blend of Pilsner malt, aromatic malt, and Belgian candi sugar. This strong ale is characterized by its golden hue, robust body, and high carbonation. It boasts a complex flavor profile with notes of fruity esters, spicy phenols, and a subtle sweetness from the candi sugar. The beer has a moderate bitterness that balances out the malt

In [21]:
def user(user_message, history):
    return "", history + [[user_message, None]]

def get_answer(search_prompt, rag_method):
    if rag_method == "Vector-Search":
        scored_beers = get_context_vector_search(search_prompt)
        theprompt = generate_prompt_vector_search(search_prompt, scored_beers)
    else: 
    # rag_method == "GraphRAG"
        scored_beers = get_context_graphrag(search_prompt)
        theprompt = generate_prompt_graphrag(search_prompt, scored_beers)
    messages = llm(theprompt.to_messages())
    return messages.content

def bot(history, rag_method):
    bot_message = get_answer(history[-1][0], rag_method)
    history[-1][1] = ""
    for character in bot_message:
        history[-1][1] += character
        time.sleep(0.01)
        yield history

with gr.Blocks() as demo:
    chatbot = gr.Chatbot(
        label="Chatbot with RAG", 
        avatar_images=["https://png.pngtree.com/png-vector/20220525/ourmid/pngtree-concept-of-facial-animal-avatar-chatbot-dog-chat-machine-illustration-vector-png-image_46652864.jpg","https://d-cb.jc-cdn.com/sites/crackberry.com/files/styles/larger/public/article_images/2023/08/openai-logo.jpg"]
    )
    msg = gr.Textbox(label="Message")
    rag_method = gr.Radio(["Vector-Search", "GraphRAG"], label="RAG-method:")
    clear = gr.Button("Clear")


    msg.submit(user, [msg, chatbot], [msg, chatbot], queue=False).then(
        bot, [chatbot, rag_method], chatbot
    )
    
    clear.click(lambda: None, None, chatbot, queue=False)

    
demo.queue()
demo.launch(share=False)

/Users/erikbijl/anaconda3/lib/python3.11/site-packages/gradio/components/chatbot.py:223: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
